In [1]:
!pip install docx2txt

  Preparing metadata (setup.py) ... - done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3980 sha256=94e01e791bf570dbfa08297f4dd3b3d821f15b791c8c16339ac9860b537605df
  Stored in directory: /root/.cache/pip/wheels/08/45/4d/5985be6d2ead508294e8660bdf11c974153d1f519e96810d0c
Successfully built docx2txt


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
from glob import glob
import gc
import docx2txt
import re

In [3]:
folder = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/*'))

## World History Volume 1

- No Testbank, Each Chapter has Review, Check your Understanding and Reflection Questions


In [4]:
wh1 = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Worldhistory_docx/world-history-volume-1/*.docx'))

In [5]:
text = docx2txt.process(wh1[7])
access_for_free = text[-41:]
text

'Review Questions\n\n1. \n\nWhat is an example of a primary source?\n\na diary entry by a person who lived in the period under discussion\n\na modern biography of a person in the period under discussion\n\nan account of a nineteenth-century battle in a twenty-first century textbook\n\nan article in an academic journal\n\n2. \n\nWhom do the Chinese view as the Father of History?\n\nHomer\n\nSantayana\n\nHerodotus\n\nSima Qian\n\n3. \n\nWhat interpretation of history assumes that history can be viewed primarily through the lives and choices of leaders?\n\ngreat man theory\n\nprogressive interpretation\n\ngender interpretation\n\nMarxist interpretation\n\n4. \n\nThe belief that history is moved primarily by class struggle is the ______ of history.\n\nsocial interpretation theory\n\nrevisionist view\n\nprogressive interpretation\n\nMarxist interpretation\n\n5. \n\nThe _______ is the most immediate motivator of a historical event.\n\ntertiary cause\n\nprimary cause\n\naction of a great man\

In [6]:
pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
qs = re.findall(pattern, text, re.DOTALL)
qs[1]

'\n\nWhom do the Chinese view as the Father of History?\n\nHomer\n\nSantayana\n\nHerodotus\n\nSima Qian\n\n3'

In [7]:
BOOK = 'World History Volume 1'

In [8]:
def extract(files, BOOK):
    rows = []
    for file in files:
        text = docx2txt.process(file)
        filename = file.split('/')[-1]
        is_question = 'questions' in filename
        section_title = re.sub(r"\.docx$", "", filename)
        if is_question:
            pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
            qs = re.findall(pattern, text, re.DOTALL)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text.strip(access_for_free)
        chapter = filename.split('-')[0]
        if is_question:
            for q in qs:
                #iterate over the questions
                fill_blank = False
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = q.strip('Access')
                if "___" in q:
                    fill_blank = True
                row['truefalse'] = False
                row['fillblank'] = fill_blank
                row['mc'] = !fill_blank
                rows.append(row)
        else:
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = False
                row['text'] = text
                rows.append(row)
    return rows

In [9]:
#tq stands for text and questions
pd.DataFrame(extract(wh1, BOOK)).to_parquet('wh1_tq.pq')
# df[df['is_question'] == True].text.iloc[30]

In [10]:
del wh1
gc.collect()

21

## World History Volume 2

- Each Chapter has Application and Reflection Questions
- We have a Testbank that is organized See next part

In [11]:
BOOK = 'World History Volume 2'

In [12]:
wh2 = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Worldhistory_docx/world-history-volume-2/*.docx'))

In [13]:
def extract2(files, BOOK):
    rows = []
    for file in files:
        text = docx2txt.process(file)
        filename = file.split('/')[-1]
        is_question = 'questions' in filename
        section_title = re.sub(r"\.docx$", "", filename)
        if is_question:
            pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
            qs = re.findall(pattern, text, re.DOTALL)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text.strip(access_for_free)
        chapter = filename.split('-')[0]
        if is_question:
            for q in qs:
                #iterate over the questions
                fill_blank = False
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = q.strip('Access')
                row['truefalse'] = False
                row['fillblank'] = "___" in q
                row['mc'] = True
                rows.append(row)
        else:
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = False
                row['text'] = text
                rows.append(row)
    return rows

In [14]:
#tq stands for text and questions
pd.DataFrame(extract2(wh2, BOOK = 'World History Volume 2')).to_parquet('wh2_tq.pq')

In [15]:
del wh2
gc.collect()

21

## WH2 Testbank
- testbank has all types of nice questions
- right now only extract fillblank or mc flag, but some of them could be more nuanced, but each file is different, FUTURE WORK!

In [16]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Worldhistory_vol2_testbank/*.docx'))

In [17]:
text = docx2txt.process(files[0])
try:
    start = re.search("\nAccess for free", text).start()
    text = text[:start]
except:
    text = text.strip(access_for_free)

In [18]:
def extract_wh2tb(files, BOOK):
    rows = []
    for file in files:
        filename = file.split('/')[-1]
        chapter = filename.strip('.docx').split('-')[-1].strip('Ch')
        chapter = int(chapter)
        text = docx2txt.process(file).strip(access_for_free)
        questions = re.findall("\n.*\?", text)
        tf = re.findall("\nTrue or false.*\n", text)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text.strip(access_for_free)
        for question in questions:
            row = dict()
            row['Book'] = BOOK
            if "True or false?" in question:
                continue
            else:            
                row['Chapter'] = chapter
                row['question'] = question.strip('\n')
                row['is_question'] = True
                if '___' in question:
                    row['fillblank'] = True
                else:
                    row['fillblank'] = False
                rows.append(row)
        for question in tf: 
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['question'] = question.strip('\n')
            row['is_question'] = True
            row['truefalse'] = True
            row['fillblank'] = False
            rows.append(row)
    return rows

In [19]:
pd.DataFrame(extract_wh2tb(files, BOOK)).fillna(False).to_parquet('wh2testbank.pq')

## Psychology 2e 
- have Testbank
- questions are marked end of chapter, critical thinking, review, personal application
- lots of Fill blank, questions are very long, maybe to psych you out XD

In [20]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/psychology-2e-docx/psychology-2e/*.docx'))
text = docx2txt.process(files[2])
access_for_free = text[-41:]
try:
    start = re.search("\nAccess for free", text).start()
    text = text[:start]
except:
    text = text.strip(access_for_free)

In [21]:
def extract_psych(files, BOOK):
    rows = []
    for file in files:
        text = docx2txt.process(file)
        filename = file.split('/')[-1]
        is_question = 'questions' in filename
        section_title = re.sub(r"\.docx$", "", filename)
        if is_question:
            pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
            qs = re.findall(pattern, text, re.DOTALL)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text.strip(access_for_free)
        chapter = filename.split('-')[0]
        if is_question:
            for q in qs:
                #iterate over the questions
                fill_blank = False
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = q.strip('Access')
                if "___" in q:
                    row['fillblank'] = True
                else:
                    row['fillblank'] = True
                row['truefalse'] = False
                row['mc'] = False
                    
                rows.append(row)
        else:
                row = dict()
                row['Book'] =BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = False
                row['text'] = text
                rows.append(row)
    return rows

In [22]:
book = 'Psychology 2e'

In [23]:
pd.DataFrame(extract_psych(files, BOOK = book)).to_parquet('psychology2e_tq.pq')

In [24]:
pd.DataFrame(extract_psych(files, BOOK = book))

,Book,Chapter,section_title,is_question,text,fillblank,truefalse,mc
0,Psychology 2e,1,1-1-what-is-psychology,False,1.1 What Is Psychology?\n\nLearning Objectiv...,NaN,NaN,NaN
1,Psychology 2e,1,1-2-history-of-psychology,False,1.2 History of Psychology\n\nLearning Object...,NaN,NaN,NaN
2,Psychology 2e,1,1-3-contemporary-psychology,False,1.3 Contemporary Psychology\n\nLearning Obje...,NaN,NaN,NaN
3,Psychology 2e,1,1-4-careers-in-psychology,False,1.4 Careers in Psychology\n\nLearning Object...,NaN,NaN,NaN
4,Psychology 2e,1,1-critical-thinking-questions,True,\n\nWhy do you think psychology courses like t...,True,False,False
...,...,...,...,...,...,...,...,...
700,Psychology 2e,9,9-review-questions,True,\n\nWhich of the following is the order of sta...,True,False,False
701,Psychology 2e,9,9-summary,False,Summary\n\n\n\n9.1 What Is Lifespan Developm...,NaN,NaN,NaN
702,Psychology 2e,index.docx,index,False,Index\n\nA\n\nabsentmindedness 8.3 Problems wi...,NaN,NaN,NaN
703,Psychology 2e,preface.docx,preface,False,"Preface\n\nWelcome to Psychology 2e, an OpenSt...",NaN,NaN,NaN


## Psych 2e TB

In [25]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Psychology_testbank/Test Bank/*.docx'))
text = docx2txt.process(files[0])
access_for_free = text[-70:] #copyright too

In [26]:
def extract_psychtb(files, book):
    rows = []
    for file in files:
        filename = file.split('/')[-1]
        chapter = filename.strip('.docx').split('_')[-1].strip('Ch')
        chapter = int(chapter)
        text = docx2txt.process(file).strip(access_for_free)
        questions = re.findall("\n.*\?", text)
        tf = re.findall("\nTrue or false.*\n", text)
        for question in questions:
            row = dict()
            #actually psych has no true or false, but leave it just in case, bc I didn't go over all files
            if "True or false?" in question:
                continue
            if "Text Section:" in question:
                continue
            else:        
                row['Book'] = book
                row['Chapter'] = chapter
                row['question'] = question.strip('\n')
                row['is_question'] = True
                row['fillblank'] = "___" in question
                row["truefalse"] = False
                rows.append(row)
        for question in tf: 
            row = dict()
            row["Book"] = book
            row['Chapter'] = chapter
            row['question'] = question.strip('\n')
            row['is_question'] = True
            row['fillblank'] = '___' in question
            row['truefalse'] = True
            rows.append(row)
    return rows

In [27]:
pd.DataFrame(extract_psychtb(files, book = book)).to_parquet('psych2e_testbank.pq')

# Business Ethics

- Questions in the docx are marked as "assessment questions"
- Includes summaries
- "chapter-xy.docx" files are solutions to the assement questions 

In [28]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/business-ethics-docx/*.docx'))
text = docx2txt.process(files[2])
access_for_free = text[-41:]
try:
    start = re.search("\nAccess for free", text).start()
    text = text[:start]
except:
    text = text.strip(access_for_free)
    
BOOK = 'Business Ethics'

In [29]:
# wwyd_start = re.search('What Would You Do', text).start()
# text[wwyd_start:critical_start]

In [30]:
pattern = r'\n\n(?:[A-Z].*?)\n\n'
qs = re.findall(pattern, text, re.DOTALL)
# qs

In [31]:
def get_question(q, chapter, section_title, book):
    row = dict()
    row['Book'] = book
    row['Chapter'] = chapter
    row['section_title'] = section_title
    row['is_question'] = True
    row['text'] = q.strip('Access')
    row['truefalse'] = False
    row['fillblank'] = "___" in q
    row['mc'] = "___" in q
    return row

In [32]:
# for file in files:
#     filename = file.split('/')[-1]
#     chapter = filename.split('-')[0]
#     section_title = re.sub(r"\.docx$", "", filename)
#     if 'endnotes' in filename:
#         print(f"filename: {filename}, chapter: {chapter}, section: {section_title}")

In [33]:
def extract_bethics(files, BOOK):
    rows = []
    for file in files:
        text = docx2txt.process(file).strip(access_for_free)
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        is_question = "questions" in filename
        if is_question:
            pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
            qs = re.findall(pattern, text, re.DOTALL)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text.strip(access_for_free)
        
        if is_question:
            for q in qs:
                #iterate over the questions
                row = get_question(q,
                                   chapter,
                                   section_title,
                                   book = BOOK
                                    )
                rows.append(row)
        else:
            try:    
                critical_start = re.search('Critical Thinking', text).start()
                critical = text[critical_start:].split('\n\n')[1:]
                text = text[:critical_start]
                try:
                    wwyd_start = re.search('What Would You Do', text).start()
                    wwyd = text[wwyd_start:critical_start]
                except:
                    wwyd = None
            except:
                text = text
                critical = []
                wwyd = None
            if wwyd != None:
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = wwyd
                row['truefalse'] = False
                row['fillblank'] = "___" in q
                row['mc'] = "___" in q
                row['open_ended'] = False
                rows.append(row)
                
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = "questions" in filename
            row['text'] = text
            row['question_answers'] = bool(re.search('chapter-[1-9]', filename))
            if row['question_answers']:
                row['Chapter'] = section_title.split('-')[-1]
            rows.append(row)
            for q in critical:
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = q.strip('Access')
                row['truefalse'] = False
                row['fillblank'] = "___" in q
                row['mc'] = "___" in q
                row['open_ended'] = True
                rows.append(row)
    return rows

In [34]:
pd.DataFrame(extract_bethics(files, BOOK = BOOK)).to_parquet('business_ethics_tq.pq')
BOOK

'Business Ethics'

# Business Ethics Testbank

- MC Questions
- Fill the blank Questions
- True or False
- Short Answer
- Have Difficulty rating


In [35]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/*.docx'))
text = docx2txt.process(files[0])
access_for_free = text[-85:] #copyright too

In [36]:
for file in files:
    print(file)

/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch01_WhyEthicsMatters.docx
/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch02_EthicsFromAntiquity.docx
/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch03_DefiningAndPrioritizingStakeholders.docx
/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch04_ThreeSpecialStakeholders.docx
/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch05_ImpactofCulture.docx
/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch06_EmployersOweEmployees.docx
/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch07_EmployeesOweEmployers.docx
/kaggle/input/csnlp-datasets/openstax/openstax/BusinessEthics-TestBank/Test Banks/OSX_EthicsTB_Ch08_Rightso

In [37]:
def extract_bethicstb(files, BOOK):
    rows = []
    for file in files:
        filename = file.split('/')[-1]
        chapter = re.sub(r"Ch", "", file.split('/')[-1].split('_')[-2])
        chapter = int(chapter)
        text = docx2txt.process(file).strip(access_for_free)
        questions = re.findall("\n.*\?", text)
        tf = re.findall("\nTrue or false.*\n", text)
        short_answers = re.findall("\n..*\n\nSample Answer:?", text)
        for sa in short_answers:
            row = dict()
            row['Book'] = BOOK
            sa = re.sub("\n\nSample Answer:", "", sa)
            row['Chapter'] = chapter
            row['question'] = sa.strip('\n')
            row['is_question'] = True
            row['open_ended'] = True
            rows.append(row)
        #creates some duplicates
        for question in questions:
            row = dict()
            #actually psych has no true or false, but leave it just in case, bc I didn't go over all files
            if "True or false?" in question:
                continue
            if "Text Section:" in question:
                continue
            else:
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['question'] = question.strip('\n')
                row['is_question'] = True
                rows.append(row)
        for question in tf: 
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['question'] = question.strip('\n')
            row['is_question'] = True
            row['truefalse'] = True
            row['fillblank'] = '___' in question
            rows.append(row)
    return rows

In [38]:
pd.DataFrame(extract_bethicstb(files, BOOK)).to_parquet('business_ethics_testbank.pq')

In [39]:
pd.DataFrame(extract_bethicstb(files, BOOK))

,Book,Chapter,question,is_question,open_ended,truefalse,fillblank
0,Business Ethics,1,Provide five examples of stakeholders.,True,True,NaN,NaN
1,Business Ethics,1,What motivated Mark Faris to be a white-collar...,True,True,NaN,NaN
2,Business Ethics,1,How are stockholders also stakeholders? Explai...,True,True,NaN,NaN
3,Business Ethics,1,"What is compliance, and how is does it operate?",True,True,NaN,NaN
4,Business Ethics,1,Explain the concept of corporate social respon...,True,True,NaN,NaN
...,...,...,...,...,...,...,...
519,Business Ethics,10,True or false? There is a prevailing view that...,True,NaN,True,False
520,Business Ethics,10,True or false? Stressed workers are less produ...,True,NaN,True,False
521,Business Ethics,10,True or false? Millennials do not believe inte...,True,NaN,True,False
522,Business Ethics,10,True or false? Flextime is a practical way to ...,True,NaN,True,False


# Philosophy

- end of chapter review questions
- end of chapter references, key terms, summary
- sometimes further reading


In [40]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/introduction-philosophy/introduction-philosophy/*.docx'))
text = docx2txt.process(files[8])
access_for_free = text[-41:] #copyright too
# access_for_free
try:
    start = re.search("\nAccess for free", text).start()
    text = text[:start]
except:
    text = text.strip(access_for_free)
book = 'Philosophy'

In [41]:

pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
qs = re.findall(pattern, text, re.DOTALL)
# qs

In [42]:
def extract_phil(files, book):
    rows = []
    for file in files:
        text = docx2txt.process(file).strip(access_for_free)
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        is_question = "questions" in filename
        if is_question:
            pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
            qs = re.findall(pattern, text, re.DOTALL)
            for q in qs:
                row = get_question(q.strip('\n\n'),
                                    chapter,
                                    section_title,
                                    book = book)
                rows.append(row)
        else:
            row = dict()
            row['Book'] = book
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = False
            row['text'] = text.strip('\n\n')
            rows.append(row)
    return rows

In [43]:
pd.DataFrame(extract_phil(files, book = book)).to_parquet('philosophy_tq.pq')

# Philosophy Testbank

- MC but not marked like in others
- True or false

In [44]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Philosophy_testbank/*.docx'))
text = docx2txt.process(files[0])
access_for_free = text[-235:] #copyright too
access_for_free

'\n\n\n\nThis file is copyright 2022, Rice University. All Rights Reserved. \n\n\n\n05/12/22                                                                                                                                                       2'

In [45]:
re.sub(r"Ch", "", files[0].split('/')[-1].split('-')[-1]).strip('.docx')

'1'

In [46]:
def extract_philtb(files, book):
    rows = []
    for file in files:
        filename = file.split('/')[-1]
        chapter = re.sub(r"Ch", "", file.split('/')[-1].split('-')[-1]).strip('.docx')
        chapter = int(chapter)
        text = docx2txt.process(file).strip(access_for_free)
        questions = re.findall("\n.*\?", text)
        tf = re.findall("\nTrue or false.*\n", text)
        short_answers = re.findall("\n..*\n\nSample Answer:?", text)
        for sa in short_answers:
            row = dict()
            sa = re.sub("\n\nSample Answer:", "", sa)
            row['Book'] = book
            row['Chapter'] = chapter
            row['question'] = sa.strip('\n')
            row['is_question'] = True
            row['open_ended'] = True
            rows.append(row)
        #creates some duplicates
        for question in questions:
            row = dict()
            #actually psych has no true or false, but leave it just in case, bc I didn't go over all files
            if "True or false?" in question:
                continue
            if "Text Section:" in question:
                continue
            else:
                row['Book'] = book
                row['Chapter'] = chapter
                row['question'] = question.strip('\n')
                row['is_question'] = True
                rows.append(row)
        for question in tf: 
            row = dict()
            row['Book'] = book
            row['Chapter'] = chapter
            row['question'] = question.strip('\n')
            row['is_question'] = True
            row['truefalse'] = True
            rows.append(row)
    return rows

In [47]:
pd.DataFrame(extract_philtb(files, book)).to_parquet('philosophy_testbank.pq')
print(book)

Philosophy


# Business Law Essentials

- assesement questions end of chapter
- answers "chapter-[1-9][1-9]" odd numbered questions only :-S
- endnotes at end of every chapter, summary and key terms basically


In [48]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Business_Law_Essentials_Docx-customizable/business-law-i-essentials/*.docx'))
text = docx2txt.process(files[2])
access_for_free = text[-41:]
access_for_free
BOOK = 'Business Law Essentials'

In [49]:
pd.DataFrame(extract_bethics(files, BOOK)).to_parquet('business_law_essentials_tq.pq')
BOOK

'Business Law Essentials'

# Business Law Testbank 
Only has instructor answers

# Entrepreneurship

- discussion, review, case questions
- end of chapter summaries, key-terms, and links to outside resources

In [50]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/entrepreneurship-docx/*.docx'))
text = docx2txt.process(files[0])
access_for_free = text[-41:] #copyright too
BOOK = 'Entrepreneurship'

In [51]:
def extract_entrep(files, book):
    rows = []
    for file in files:
        row = dict()
        text = docx2txt.process(file).strip(access_for_free)
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        is_question = "questions" in filename
        if is_question:
            pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
            qs = re.findall(pattern, text, re.DOTALL)
            for q in qs:
                row = get_question(q.strip('\n\n'),
                                    chapter,
                                    section_title,
                                    book = book)
                rows.append(row)
        else:
            row = dict()
            row['Book'] = book
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = False
            row['text'] = text.strip('\n\n')
            rows.append(row)
    return rows

In [52]:
pd.DataFrame(extract_entrep(files, book = BOOK)).to_parquet('entrepreneurship.pq')
print(BOOK)

Entrepreneurship


# Entrepreneurship Testbank

- MC/Fill in the blank
- True or false
- Short Answer

In [53]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/EShip-TestBanks/Test Banks/*.docx'))
text = docx2txt.process(files[2])
access_for_free = text[-235:] #copyright too
access_for_free

' his seventies. Yunus is known for supporting women as micro-entrepreneurs, and 97 percent of Grameen bank borrowers are women.\n\n\n\nDifficulty: Moderate\n\n\n\nThis file is copyright 2020, Rice University. All Rights Reserved.\n\nPage 1 of 12'

In [54]:
def extract_enttb(files, BOOK):
    rows = []
    for file in files:
        filename = file.split('/')[-1]
        chapter = re.sub(r"Ch", "", file.split('/')[-1].split('_')[-2])
        chapter = int(chapter)
        text = docx2txt.process(file).strip(access_for_free)
        questions = re.findall("\n.*\?", text)
        tf = re.findall("\nTrue or false.*\n", text)
        short_answers = re.findall("\n..*\n\nSample Answer:?", text)
        for sa in short_answers:
            row = dict()
            row['Book'] = BOOK
            sa = re.sub("\n\nSample Answer:", "", sa)
            row['Chapter'] = chapter
            row['question'] = sa.strip('\n')
            row['is_question'] = True
            row['open_ended'] = True
            rows.append(row)
        #creates some duplicates
        for question in questions:
            row = dict()
            #actually psych has no true or false, but leave it just in case, bc I didn't go over all files
            if "True or false?" in question:
                continue
            if "Text Section:" in question:
                continue
            else:
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['question'] = question.strip('\n')
                row['is_question'] = True
                rows.append(row)
        for question in tf: 
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['question'] = question.strip('\n')
            row['is_question'] = True
            row['truefalse'] = True
            row['fillblank'] = '___' in question
            rows.append(row)
    return rows

In [55]:
pd.DataFrame(extract_enttb(files, BOOK = BOOK)).to_parquet('entrepreneurship_testbank.pq')
print(BOOK)

Entrepreneurship


# Introduction to Business
- questions are in a separate file, though most sections have a concept check which is questions, they are always at the end of the file


In [56]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Docx-customizable/introduction-business/*.docx'))
text = docx2txt.process(files[2])
access_for_free = text[-41:] #copyright too
book = 'Introduction to Business'

In [57]:
re.findall('\n\nConcept Check(.*?)\n\nA', text, re.DOTALL)

['\n\nWhat is economics, and how can you benefit from understanding basic economic concepts?\n\nCompare and contrast the world’s major economic systems. Why is capitalism growing, communism declining, and socialism still popular?\n\nWhat is the difference between macroeconomics and microeconomics?']

In [58]:
def extract_introbiz(files, book):
    rows = []
    for file in files:
        
        text = docx2txt.process(file)
        try:
#             print(f"have questions in: {file}")
            conceptchecks = re.findall('\n\nConcept Check(.*?)\n\nA', text, re.DOTALL)[0].split('\n\n')#before removing access because I need it to match
        except:
            conceptchecks = []
#             print(f"no questions in {file}")
        text = text.strip(access_for_free)
        try:
            end = re.search('\n\nConcept Check', text, re.DOTALL).start()
            text = text[:end]
        except:
            print(f'no concept check in {file}')
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        is_question = 'questions' in filename
        if is_question:
            pattern = r'\n\n(?:[A-Z].*?)\n\n(?:\d|Access)'
            qs = re.findall(pattern, text, re.DOTALL)
            for q in qs:
                row = get_question(q.strip('\n\n'),
                                    chapter,
                                    section_title,
                                    book = book)
                rows.append(row)
        else:    
            row = dict()
            row['Book'] = book
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = "questions" in filename
            row['text'] = text
            rows.append(row)
            for cc in conceptchecks:
                if len(cc) == 0:
                    continue
                row = dict()
                row['Book'] = book
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = cc
                rows.append(row)
    return rows

In [59]:
pd.DataFrame(extract_introbiz(files, book)).to_parquet('intro2biz_tq.pq')
print(book)

no concept check in /kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Docx-customizable/introduction-business/1-critical-thinking-case.docx
no concept check in /kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Docx-customizable/introduction-business/1-ethics-activity.docx
no concept check in /kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Docx-customizable/introduction-business/1-hot-links-address-book.docx
no concept check in /kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Docx-customizable/introduction-business/1-introduction.docx
no concept check in /kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Docx-customizable/introduction-business/1-key-terms.docx
no concept check in /kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Docx-customizable/introduction-business/1-preparing-for-tomorrows-workplace-skills.docx
no concept check in /kaggle/input/csnlp-

# Review Questions
- this one is actually very nice, each section has review questions and they are ordered

In [60]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Review_Questions/Review Questions/*.docx'))
text = docx2txt.process(files[3])
# access_for_free = text[-605:] #copyright too
# access_for_free
book

'Introduction to Business'

In [61]:
start = re.search("\nTo answer a particular question", text).start()
access_for_free = text[start:] #copyright too
access_for_free

'\nTo answer a particular question, visit the respective chapter section reading in the OpenStax Introduction to Business textbook. Questions were derived from textbook content: https://openstax.org/details/books/introduction-business\n\n\n\nThis OpenStax ancillary resource is © Rice University under a CC BY 4.0 International license; it may be reproduced or modified but must be attributed to OpenStax, Rice University and any changes must be noted.\n\nFor more free, peer-reviewed, openly licensed resources visit OpenStax.org.'

In [62]:
re.findall('[1-9]\.[1-9] Review Questions', text) #word :-/

['7.1 Review Questions',
 '7.3 Review Questions',
 '7.4 Review Questions',
 '7.6 Review Questions',
 '7.7 Review Questions']

In [63]:
re.findall('Review Questions(.*?)\n[1-9]', text, re.DOTALL)[0].split('\n\n')

['',
 'Define the following terms',
 'Organization',
 'Division of Labor',
 'Specialization',
 'Departmentalization',
 'Organizational Chart',
 'What are the 3 organizing or structuring processes?',
 'Which organizing process determines work activities and divides up tasks?',
 'Which organizing process focuses on grouping jobs and employees?',
 'What does delegation do?',
 'What is an organization?',
 'Division of Labor',
 'What are traditional organization structures? ',
 'What does departmentalization facilitate?',
 'What is the purpose of an organizational chart?',
 'What are the 5 basic types of departmentalization?',
 'Which form of departmentalization is based on the primary functions performed within an organizational unit?',
 'What type of organization might use functional departmentalization?',
 'What is product departmentalization based on, and what types of organizations would this work well for?',
 'What types of organizations would benefit from process departmentalization?

In [64]:
files[8].split('/')[-1].strip('.docx').split('Ch ')[-1]

'12'

In [65]:
def extract_introbiz_questions(files, book):
    rows = []
#     book = 'Introduction to Business'
    for file in files:
        text = docx2txt.process(file)
        start = re.search("\nTo answer a particular question", text).start()
        access_for_free = text[start:] #here not all are the same
        text = text.strip(access_for_free)
        review_questions = re.findall('Review Questions(.*?)\n[1-9]', text, re.DOTALL)
        for i, qs in enumerate(review_questions):
            filename = file.split('/')[-1]
            section_title = re.sub(r"\.docx$", "", filename)
            chapter = section_title.split('Ch ')[-1]
            for q in qs.split('\n\n'):
                if len(q) == 0:
                    continue
                row = dict()
                row['Book'] = book
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = q
#                 row['section'] = str(chapter).strip('0') + "." + str(i + 1)/
                rows.append(row)
    return rows

In [66]:
pd.DataFrame(extract_introbiz_questions(files, book)).to_parquet('intro2biz_review_questions.pq')
pd.DataFrame(extract_introbiz_questions(files, book)).sample(10)

,Book,Chapter,section_title,is_question,text
1141,Introduction to Business,14,Introduction to Business Review Questions Ch 14,True,What are the two types of financial reports?
1176,Introduction to Business,14,Introduction to Business Review Questions Ch 14,True,What are the seps involved in finding net prof...
302,Introduction to Business,07,Introduction to Business Review Questions Ch 07,True,Who are problem-solving teams made up of?
103,Introduction to Business,05,Introduction to Business Review Questions Ch 05,True,How many people do most established small busi...
170,Introduction to Business,06,Introduction to Business Review Questions Ch 06,True,Why is it important for organizations to antic...
844,Introduction to Business,11,Introduction to Business Review Questions Ch 11,True,Shopping Product
1661,Introduction to Business,03,Introduction to BusinessReviewQuestionsCh 03,True,Define the following terms:
15,Introduction to Business,04,Introduction to Business Review Questions Ch 04,True,"Before forming a partnership, what disadvantag..."
383,Introduction to Business,08,Introduction to Business Review Questions Ch 08,True,Why is it important for applicants to carefull...
1644,Introduction to Business,03,Introduction to BusinessReviewQuestionsCh 03,True,What is a bilateral trade agreement?


# Intro2Biz Testbank
- this one even has a testbank


In [67]:
files[2]

'/kaggle/input/csnlp-datasets/openstax/openstax/Introduction_to_Business_Review_Questions/Review Questions/Introduction to Business Review Questions Ch 06.docx'

In [68]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/IntroductiontoBusiness-Test_Bank/Test Bank/*.docx'))
text = docx2txt.process(files[2])
# text

In [69]:
def extract_biztb(files, book):
    rows = []
    for file in files:
        filename = file.split('/')[-1].strip('.docx')
        chapter = filename.split('-')[-1].strip('Ch')
        questions = re.findall('[1-9]\.(.*)\n', text)
        for q in questions:
            if 'OpenStax' in q:
                continue
            row = dict()
            row['Book'] = book
            row['Chapter'] = chapter
            row['text'] = q
            row['is_question'] = True
            rows.append(row)
    return rows

In [70]:
pd.DataFrame(extract_biztb(files, book)).to_parquet('intro2biz_testbank.pq')

In [71]:
pd.DataFrame(extract_biztb(files, book)).sample(10)

,Book,Chapter,text,is_question
955,Introduction to Business,11,Which of the following statements about utili...,True
827,Introduction to Business,09,Strategic giving:,True
1276,Introduction to Business,15,Human rights include an individual’s right to...,True
447,Introduction to Business,05,Mamma's Diner was a small restaurant that was...,True
719,Introduction to Business,08,At the base of the corporate social responsib...,True
688,Introduction to Business,08,Social responsibility is a voluntary obligation.,True
414,Introduction to Business,04,The lowest level on the pyramid of social res...,True
889,Introduction to Business,10,At the base of the corporate social responsib...,True
761,Introduction to Business,08,"According to Peter Drucker, what are the basi...",True
1202,Introduction to Business,14,It is a firm's responsibility to its customer...,True


# Introduction to Business Statistics
- Questions are directly in text, some are quantitative linked to tables. Unsure how to process this
- Good Questions are in Homework and Instructor Solutions manual, however, would need clean the text first I presume

In [72]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Introductory-Business-Statistics_Docx-customization/Introductory-Business-Statistics_Docx-customization/*.docx'))
text = docx2txt.process(files[2])
book = 'Business Statistics'

In [73]:
def extract_biz_stats(files, book):
    
    rows = []
    skip = ['homework', 'practice', 'references', 'solutions', 'key-terms', 'index', 'preface']
    for file in files:
        filename = file.split('/')[-1].strip('.docx')
        if any([s for s in skip if s in filename]):
            continue
        else:
            text = docx2txt.process(file)
            start = re.search("\n\nAccess", text).start()
            text = text[:start]
            chapter = filename.split('-')[0]
            row = dict()
            row['Book'] = book
            row['Chapter'] = chapter
            row['section_title'] = filename
            row['text'] = text
            row['is_question'] = False
            rows.append(row)
    return rows

In [74]:
pd.DataFrame(extract_biz_stats(files, book)).to_parquet('bizstatistics_texts.pq')
pd.DataFrame(extract_biz_stats(files, book)).sample(10)

,Book,Chapter,section_title,text,is_question
38,Business Statistics,13,13-introduction,Figure 13.1 Linear regression and correlati...,False
55,Business Statistics,3,3-formula-review,Formula Review\n\n\n\n3.1 Terminology\n\nA a...,False
63,Business Statistics,4,4-introduction,Figure 4.1 You can use probability and disc...,False
75,Business Statistics,6,6-introduction,Figure 6.1 If you ask enough people about t...,False
98,Business Statistics,b,b-mathematical-phrases-symbols-and-formulas,"Appendix B Mathematical Phrases, Symbols, a...",False
60,Business Statistics,4,4-4-poisson-distribution,4.4 Poisson Distribution\n\nAnother useful p...,False
76,Business Statistics,7,7-1-the-central-limit-theorem-for-sample-means,7.1 The Central Limit Theorem for Sample Mea...,False
80,Business Statistics,7,7-chapter-review,Chapter Review\n\n\n\n7.1 The Central Limit ...,False
3,Business Statistics,1,1-chapter-review,Chapter Review\n\n\n\n1.1 Definitions of Sta...,False
34,Business Statistics,13,13-5-interpretation-of-regression-coefficients...,13.5 Interpretation of Regression Coefficien...,False


# Biz Stats Instructor Answers Guide
- Basically all Question and Answer pairs
- very well organized
- Chapter 12 has empty exercises (also in files) it's probably incomplete, should remove


In [75]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Introductory_Business_Statistics_Instructor_Answer_Guide_WCI1hPS/Introductory Business Statistics Instructor Answer Guide/*.docx'))
text = docx2txt.process(files[2])
# text

In [76]:
re.search("\nCopyright", text).start()

81780

In [77]:
files[0].split('/')[-1].strip('.docx').split('-')[-1].strip('Ch')

'01'

In [78]:
re.findall('Exercise [1-9](.*?)\n\nSolution', text, re.DOTALL)[1]

'.\n\nA box is filled with several party favors. It contains 12 hats, 15 noisemakers, ten finger traps, and five bags of confetti. One party favor is chosen from the box at random.\n\nLet H = the event of getting a hat.\n\nLet N = the event of getting a noisemaker.\n\nLet F = the event of getting a finger trap.\n\nLet C = the event of getting a bag of confetti.\n\nFind P(H).'

In [79]:
def extract_bizstats_questions(files, book):
    print(f"book: {book}")
    rows = []
#     book = 'Business Statistics'
    for file in files:
        filename = file.split('/')[-1].strip('.docx')
        text = docx2txt.process(file)
        try:
            start = re.search("\nCopyright", text).start()
            text = text[:start]
        except:
            text = text
        chapter = filename.split('-')[-1].strip('Ch')
        questions = re.findall('Exercise [1-9](.*?)\n\nSolution', text, re.DOTALL)
        for q in questions:
            row = dict()
            row['Book'] = book
            row['Chapter'] = chapter
            row['section_title'] = filename
            row['text'] = q
            row['is_question'] = True
            rows.append(row)
    return rows

In [80]:
pd.DataFrame(extract_bizstats_questions(files, book)).to_parquet('bizstats_questions.pq')

book: Business Statistics


# Organizational Behaviour
- has one file with questions for each chapter "review questions"
- has one "critical thinking case" for each chapter, with a case study and questions, (is the whole file a question or should split?)
- some chapers have "managerial decision exercises" and "managerial skills application exercises" which are exercises and questions
- every section has a "concept check" with 2 questions

In [81]:
files[5]

'/kaggle/input/csnlp-datasets/openstax/openstax/Introductory_Business_Statistics_Instructor_Answer_Guide_WCI1hPS/Introductory Business Statistics Instructor Answer Guide/BusinessStatistics-IAG-Ch06.docx'

In [82]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/organizational-behavior/organizational-behavior/*.docx'))
text = docx2txt.process(files[5])
# text

In [83]:
cq = re.findall('Critical Thinking Case\n\n(.*?)$', text, re.DOTALL)[0]
# qs = re.findall('*?Questions\n\n(.*?)$', cq, re.DOTALL)#[0].split('\n\n')
# cq

In [84]:
cq[re.search('\nQuestions:', cq, re.DOTALL).end():].split('\n\n')

['',
 'How has the current cultural environment of our country shaped the way that companies are looking at their own corporate cultural standards?',
 'What are the potential downfalls and positive influences of the “Netflix way”?',
 'How does Netflix’s internal culture negatively or positively affect their ability to stay competitive and deliver cutting-edge content?',
 'Sources: B. Stelter, “The Weinstein Effect: Harvey Weinstein scandal sparks movements in Hollywood and beyond,” CNN Business, October 20, 2017, https://money.cnn.com/2017/10/20/media/weinstein-effect-harvey-weinstein/; L. Hertzler, “Talking #MeToo, one year after bombshell Weinstein allegations,” Penn Today, October 30, 2018, https://penntoday.upenn.edu/news/talking-me-too-one-year-later; S. Ramachandaran and J. Flint, “At Netflix, Radical Transparency and Blunt Firings Unsettle the Ranks,” Wall Street Journal, October 25, 2018, https://www.wsj.com/articles/at-netflix-radical-transparency-and-blunt-firings-unsettle-th

In [85]:
# re.findall('Review Questions\n\n(.*?)$', text, re.DOTALL)[0].split('\n\n')
# # re.findall('\n\n.*Questions\n\n(.*?)$', cq, re.DOTALL)[0].split('\n\n')
BOOK = 'Organizational Behaviour'

In [86]:
question_files = ['critical-thinking-case', 'questions', 'exercises']
def extract_ob_tq(files, BOOK):
    print(f"book: {BOOK}")
    rows = []
    for file in files:
        row = dict()
        text = docx2txt.process(file)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        is_question = any(q for q in question_files if q in file)
        if is_question:
            if 'exercises' in filename and 'managerial' in filename:
                qs = re.findall('Managerial Decision Exercises\n(.*?)$', text, re.DOTALL)[0].split('\n\n')
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            elif 'exercises' in filename and 'skills' in filename:
                qs = re.findall('Management Skills Application Exercises\n(.*?)$', text, re.DOTALL)[0].split('\n\n')
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            elif 'critical-thinking-case' in filename:
                try:
                    cq = re.findall('Critical Thinking Case\n\n(.*?)$', text, re.DOTALL)[0]
                    qs = re.findall('\n\n.*Questions\n\n(.*?)$', cq, re.DOTALL)[0].split('\n\n')
#                     print(f"success:{qs} \n\n {cq} \n\n")
                except:
                    start = re.search('Questions:', cq).end()
                    end = re.search('Sources:', cq).start()
                    qs = cq[start:end].split('\n\n')
#                     print(f"Critical Thinking: {filename}")
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    if len(q) == 0:
                        continue
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = False
                row['text'] = q.strip('\n\n').strip('\n')
                rows.append(row)
            # Last case review Questions
            elif 'questions' in filename:
                qs = re.findall('Review Questions\n\n(.*?)$', text, re.DOTALL)[0].split('\n\n')
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            else:
                print(f"this file is missing: {filename}")
                continue
        else:
#             concepts = re.findall('Concept Check\n\n.*?$', text, re.DOTALL)[0].split('\n\n')[1:]
            try:
                start = re.search("\nConcept Check", text).start()
                concepts = text[start:].split('\n\n')[1:]
                text = text[:start]
            except:
                concepts = []
            for concept in concepts:
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = concept #text.strip('\n\n').strip('\n')
                rows.append(row)
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = False
            row['text'] = text.strip('\n\n').strip('\n')
            rows.append(row)
    return rows

In [87]:
pd.DataFrame(extract_ob_tq(files, BOOK)).to_parquet('organizational_behaviour_tq.pq')

book: Organizational Behaviour
this file is missing: b-scoring-keys-for-self-assessment-exercises.docx


# Organizational Behaviour Testbank

-

In [88]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/OrganizationalBehaviour-TestBank/OrgB-TestBank/*.docx'))
text = docx2txt.process(files[0])
# text

In [89]:
tf = re.findall(r'LEARNING OUTCOME: [1-9](.*?)a\. True', text, re.DOTALL)

In [90]:
mcfb = re.findall(r'LEARNING OUTCOME:.*?E\..*?\n\n', text, re.DOTALL)[1:]

In [91]:
files[5].split('/')[-1].strip('.docx').split(' ')[-1]

'06'

In [92]:
def extract_ob_testbank(files, BOOK):
    print(f"book: {BOOK}")
    rows = []
    for file in files:
        filename = file.split('/')[-1].strip('.docx')
        chapter = filename.split(' ')[-1]
        start = re.search("\nThis file is copyright", text).start()
        text[:start]
        tf = re.findall(r'LEARNING OUTCOME: [1-9](.*?)a\. True', text, re.DOTALL)
        mcfb = re.findall(r'LEARNING OUTCOME:.*?E\..*?\n\n', text, re.DOTALL)[1:]
        for q in tf:
            q = re.sub(r"LEARNING OUTCOME: [1-9]", "", q)
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['text'] = q.strip('\n\n').strip('\n')
            row['is_question'] = True
            row['truefalse'] = True
            row['fillblank'] = False
            row['mc'] = False
            rows.append(row)
        for q in mcfb:
            q = re.sub(r"LEARNING OUTCOME: [1-9]", "", q)
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['text'] = q.strip('\n\n').strip('\n')
            row['is_question'] = True
            row['truefalse'] = False
            fb = 0
            if "___" in q:
                fb += 1
            row['fillblank'] = bool(fb)
            row['mc'] = bool(fb)
            rows.append(row)
    return rows

In [93]:
pd.DataFrame(extract_ob_testbank(files, BOOK)).to_parquet('organizational_behaviour_testbank.pq')

book: Organizational Behaviour


# Accounting 1 (Principles of Financial Accounting

- has lots of exercises "exercise-set" and "problem-set" for each chapter
- has multiple choice for each chapter
- answers under chapter-[1-9] files

In [94]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/principles-financial-accounting/principles-financial-accounting/*.docx'))
text = docx2txt.process(files[18])
# text

In [95]:
BOOK = 'Accounting 1'

In [96]:
access_for_free

'\nTo answer a particular question, visit the respective chapter section reading in the OpenStax Introduction to Business textbook. Questions were derived from textbook content: https://openstax.org/details/books/introduction-business\n\n\n\nThis OpenStax ancillary resource is © Rice University under a CC BY 4.0 International license; it may be reproduced or modified but must be attributed to OpenStax, Rice University and any changes must be noted.\n\nFor more free, peer-reviewed, openly licensed resources visit OpenStax.org.'

In [97]:
problems = re.findall('\n\nLO  (.*?)\n\n[1-9]\.', text, re.DOTALL)
re.sub('[1-9][0-9]\.[1-9]', '', problems[0])

'If a company has four lots of products for sale, purchase 1 (earliest) for $17, purchase 2 (middle) for $15, purchase 3 (middle) for $12, and purchase 4 (latest) for $14, which cost would be assumed to be sold first using LIFO costing?\n\n$17\n\n$15\n\n$12\n\n$14'

In [98]:
def accounting1(files, BOOK):
    print(BOOK)
    rows = []
    questions = ['problem-set', 'exercise-set', 'multiple-choice']
    for file in files:
        row = dict()
        text = docx2txt.process(file).strip(access_for_free)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        is_question = any(s for s in questions if s in filename)
        if is_question:
            if 'problem-set' in filename:
                problems = re.findall('\n\nLO  (.*?)\n\nPA', text, re.DOTALL)
                for problem in problems:
                    row = dict()
                    row['Book'] = BOOK
                    if int(chapter) > 9:
                        problem = re.sub('[1-9][0-9]\.[1-9]', '', problem)
                    else:
                        problem = re.sub('[1-9]\.[1-9]', '', problem)
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = is_question
                    row['text'] = problem
                    row['question_answers'] = False
                    rows.append(row)
            elif 'exercise-set' in filename:
                problems = re.findall('\n\nLO  (.*?)\n\nEA', text, re.DOTALL)
                for problem in problems:
                    row = dict()
                    row['Book'] = BOOK
                    if int(chapter) > 9:
                        problem = re.sub('[1-9][0-9]\.[1-9]', '', problem)
                    else:
                        problem = re.sub('[1-9]\.[1-9]', '', problem)
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = is_question
                    row['text'] = problem
                    row['question_answers'] = False
                    rows.append(row)
            elif 'multiple-choice' in filename:
                problems = re.findall('\n\nLO  (.*?)\n\n[1-9]\.', text, re.DOTALL)
                for problem in problems:
                    row = dict()
                    row['Book'] = BOOK
                    if int(chapter) > 9:
                        problem = re.sub('[1-9][0-9]\.[1-9]', '', problem)
                    else:
                        problem = re.sub('[1-9]\.[1-9]', '', problem)
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = is_question
                    row['text'] = problem
                    row['question_answers'] = False
                    rows.append(row)
        else:
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = is_question
            row['text'] = text
            row['question_answers'] = bool(re.search('chapter-[1-9]', filename))
            if row['question_answers']:
                row['Chapter'] = section_title.split('-')[-1]
            rows.append(row)
    return rows

In [99]:
pd.DataFrame(accounting1(files, BOOK)).to_parquet('accounting1_tq.pq')

Accounting 1


# Accounting 2 (Managerial Accounting)
- same format as Accounting 1


In [100]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/principles-managerial-accounting/principles-managerial-accounting/*.docx'))
text = docx2txt.process(files[18])
BOOK = 'Accounting 2'

In [101]:
pd.DataFrame(accounting1(files, BOOK)).to_parquet('accounting2_tq.pq')

Accounting 2


# Finance
- review questions and multiple choice questions in separate files for each chapter
- no questions in text
- there are video questions I skip them 

In [102]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/principles-finance-docx/*.docx'))
text = docx2txt.process(files[9])
start = re.search("\nAccess for free", text).start()
text = text[:start]
BOOK = 'Finance'

In [103]:
file = files[0]
filename = file.split('/')[-1]
section_title = re.sub(r"\.docx$", "", filename)
chapter = filename.split('-')[0]
print(filename)
print(chapter)
print(section_title)

1-1-what-is-finance.docx
1
1-1-what-is-finance


In [104]:
def finance(files, BOOK):
    rows = []
    questions = ['problem-set', 'questions', 'multiple-choice']
    for file in files:
        #questions related to a video students watch
        if 'video-activity' in file:
            continue
        text = docx2txt.process(file)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text
        
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
#         print(section_title)
        chapter = filename.split('-')[0]
        is_question = any(s for s in questions if s in filename)
        if is_question:
            if 'questions' in filename:
                problems = re.findall('\n\n[1-9]\. \n\n(.*?)\n\n', text, re.DOTALL) + re.findall('\n\n([A-Z].*?)\n\n', text, re.DOTALL)
                
                for problem in problems:
                    row = dict()
                    row['Book'] = BOOK
#                     print(problem)
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = is_question
                    row['text'] = problem
                    row['question_answers'] = False
                    row['mc_option'] = False
                    row['fillblank'] = False
                    row['mc'] = False
                    rows.append(row)
                    
            elif 'multiple-choice' in filename:
                problems = re.findall('\n\n((?:[A-Za-z]|_|[a-z]).*?)\n\n', text, re.DOTALL)
                for i, problem in enumerate(problems):
                    row = dict()
                    row['Book'] = BOOK
#                     print(problem)
                    if i % 3 == 0:      
                        row['Chapter'] = chapter
                        row['section_title'] = section_title
                        row['is_question'] = True
                        row['text'] = problem
                        row['question_answers'] = False
                        fb = 0
                        if "___" in problem:
                            fb += 1
                        row['fillblank'] = bool(fb)
                        row['mc_option'] = False
                        row['mc_num'] = i+1
                        row['mc'] = True
                        rows.append(row)
                    else:
                        
                        row['Chapter'] = chapter
                        row['section_title'] = section_title
                        row['is_question'] = is_question
                        row['text'] = problem
                        row['question_answers'] = False
                        row['mc_option'] = True
                        row['mc_num'] = i + 1
                        row['mc'] = True
                        rows.append(row)
        else:
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = is_question
            row['text'] = text
            row['question_answers'] = bool(re.search('chapter-[1-9]', filename))
            if row['question_answers']:
                row['Chapter'] = section_title.split('-')[-1]
            rows.append(row)
    return rows

In [105]:
pd.DataFrame(finance(files, BOOK)).to_parquet('finance_tq.pq')

In [106]:
df = pd.DataFrame(finance(files, BOOK))
df.head(50)

,Book,Chapter,section_title,is_question,text,question_answers,fillblank,mc_option,mc_num,mc
0,Finance,1,1-1-what-is-finance,False,1.1 What Is Finance?\n\nLearning Outcomes\n\...,False,NaN,NaN,NaN,NaN
1,Finance,1,1-2-the-role-of-finance-in-an-organization,False,1.2 The Role of Finance in an Organization\n...,False,NaN,NaN,NaN,NaN
2,Finance,1,1-3-importance-of-data-and-technology,False,1.3 Importance of Data and Technology\n\nLea...,False,NaN,NaN,NaN,NaN
3,Finance,1,1-4-careers-in-finance,False,1.4 Careers in Finance\n\nLearning Outcomes\...,False,NaN,NaN,NaN,NaN
4,Finance,1,1-5-markets-and-participants,False,1.5 Markets and Participants\n\nLearning Out...,False,NaN,NaN,NaN,NaN
5,Finance,1,1-6-microeconomic-and-macroeconomic-matters,False,1.6 Microeconomic and Macroeconomic Matters\...,False,NaN,NaN,NaN,NaN
6,Finance,1,1-7-financial-instruments,False,1.7 Financial Instruments\n\nLearning Outcom...,False,NaN,NaN,NaN,NaN
7,Finance,1,1-8-concepts-of-time-and-value,False,1.8 Concepts of Time and Value\n\nLearning O...,False,NaN,NaN,NaN,NaN
8,Finance,1,1-key-terms,False,Key Terms\n\nbrokers\n\nindividuals or a firm ...,False,NaN,NaN,NaN,NaN
9,Finance,1,1-multiple-choice,True,Which of the following was NOT identified by y...,False,False,False,1.0,True


# Finance Testbank
- Have multiple choice
- True or False
- Problems

In [107]:
length = 11
len('IMPRESSIONIST')

13

In [108]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Finance_TestBank/Test Bank/*.docx'))
text = docx2txt.process(files[5])
# BOOK = ''

In [109]:
tf = re.findall('\n\n[A-Za-z].*?\.\n\nTrue\n\nFalse', text, re.DOTALL)
tf

['\n\nTEST BANK\n\n\n\n\n\n\n\n\n\nTEST BANKs\n\n\n\n\n\n\nChapter 14\n\nTest Bank\n\nRegression Analysis in Finance\n\nThese questions and problems can all be solved in a classroom setting using a handheld financial calculator.\n\nTrue or False Questions\n\nThe numerical value of a correlation coefficient must always fall in the interval from 0 to 1 inclusive.\n\nTrue\n\nFalse',
 '\n\nSection 14.1 LO 1 Easy\n\nThe residual of a linear regression equation is calculated by subtracting the predicted y value from the observed y value, which can only produce a positive result.\n\nTrue\n\nFalse',
 '\n\nSection 14.2 LO 1 Moderate\n\nThe slope of a linear regression line and its correlation coefficient will always have the same sign. Either both will be positive or both will be negative.\n\nTrue\n\nFalse']

In [110]:
problem_end = re.search("\n\nProblems\n\n", text).end()
problem_start = re.search("\n\nProblems\n\n", text).start()
mc_start = re.search('\n\nMultiple Choice Questions\n\n', text, re.DOTALL).start()
problem_text = text[problem_end:]
problems = re.findall('\n\n.*?\n\nSection ', problem_text, re.DOTALL)
mc = re.findall('\n\n.*?\n\nSection ', text[mc_start:problem_start], re.DOTALL)
mc[4]

'\n\nThe chief financial officer of a company determines that there is a strong positive correlation between the company’s stock price and the Russell 2000 index. Which of the following is a valid interpretation of this correlation?\n\nAn increase in the Russell 2000 index will cause a decrease in the stock price.\n\nAn increase in the Russell 2000 index will cause an increase in the stock price.\n\nAs the Russell 2000 index increases, the stock price tends to decrease.\n\nAs the Russell 2000 index increases, the stock price tends to increase.\n\nSection '

In [111]:
def finance_tb(files, BOOK):
    print(f"Book: {BOOK}")
    rows = []
    for file in files:
        filename = file.split('/')[-1].strip('.docx')
        text = docx2txt.process(file)
        start = re.search("\tThis file is copyright", text).start()
        try:
            text = text[:start]
        except:
            try:
                start = re.search("\n\nTest Bank\n", text).start()
                text = text[start:]
            except:
                text = text
        text = text.strip("\n\nTest Bank")
        chapter = filename.split('Ch')[-1]
        tf = re.findall('\n\n[A-Za-z].*?\.\n\nTrue\n\nFalse', text, re.DOTALL)
        try:
            problem_end = re.search("\n\nProblems\n\n", text).end()
            problem_start = re.search("\n\nProblems\n\n", text).start()
            problem_text = text[problem_end:]
            problems = re.findall('\n\n.*?\n\nSection ', problem_text, re.DOTALL)
            try:
                mc_start = re.search('\n\nMultiple Choice Questions\n\n', text, re.DOTALL).start()
                mc = re.findall('\n\n.*?\n\nSection ', text[mc_start:problem_start], re.DOTALL)
            except:
                print("no mc questions")
        except:
            problems = []
            mc_start = re.search('\n\nMultiple Choice Questions\n\n', text, re.DOTALL).start()
            mc = re.findall('\n\n.*?\n\nSection ', text[mc_start:], re.DOTALL)

        #go over the problems
        for problem in problems:
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = True
            row['text'] = problem
            row['question_answers'] = False
            row['truefalse'] = 0
            fb = 0
            if "___" in q:
                fb += 1
            row['fillblank'] = bool(fb)
            row['mc'] = False
            rows.append(row)
        for q in mc:
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = True
            row['text'] = q
            row['question_answers'] = False
            row['truefalse'] = 0
            fb = 0
            if "___" in q:
                fb += 1
            row['fillblank'] = bool(fb)
            row['mc'] = True
            rows.append(row)
        for q in tf:
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = True
            row['text'] = q
            row['question_answers'] = False
            row['truefalse'] = 1
            fb = 0
            if "___" in q:
                fb += 1
            row['fillblank'] = bool(fb)
            row['mc'] = False
            rows.append(row)
    return rows

In [112]:
pd.DataFrame(finance_tb(files, BOOK)).to_parquet('finance_testbank.pq')

Book: Finance
no mc questions
no mc questions
no mc questions


In [113]:
pd.DataFrame(finance_tb(files, BOOK)).sample(10)

Book: Finance
no mc questions
no mc questions
no mc questions


,Book,Chapter,section_title,is_question,text,question_answers,truefalse,fillblank,mc
434,Finance,7,1-1-what-is-finance,True,"\n\nWhat is the present value of $2,350 in 5 y...",False,0,False,False
92,Finance,12,1-1-what-is-finance,True,\n\nBecause a majority of bonds pay a fixed ra...,False,0,True,True
244,Finance,18,1-1-what-is-finance,True,\n\nTEST BANK\n\n\n\n\n\n\n\n\n\nTEST BANKs\n\...,False,1,False,False
6,Finance,1,1-1-what-is-finance,True,\n\nWhich type of financial data do companies ...,False,0,False,True
237,Finance,18,1-1-what-is-finance,True,"\n\nLambda Company, a merchandising company, h...",False,0,False,True
14,Finance,1,1-1-what-is-finance,True,\n\nWhich US federal agency is responsible for...,False,0,False,True
460,Finance,7,1-1-what-is-finance,True,\n\nAn extract from a restaurant supply compan...,False,0,False,True
412,Finance,6,1-1-what-is-finance,True,\n\nMultiple Choice Questions\n\nA firm's mana...,False,0,False,True
11,Finance,1,1-1-what-is-finance,True,\n\nWhen a company’s security is sold in the s...,False,0,False,True
284,Finance,2,1-1-what-is-finance,True,\n\nWhich of the following functions does a co...,False,0,False,True


# Principles of Management
- concept checks at every section
- critical thinking case some chapters
- review questions every chapter
- managerial skills and managerial decision exercises
- has separate review questions folder with extra questions :yay:



In [114]:
for i, file in enumerate(files):
    print(f"file: {i}, {file.split('/')[-1]}")

file: 0, PrinciplesofFinance-TestBank-Ch1.docx
file: 1, PrinciplesofFinance-TestBank-Ch10.docx
file: 2, PrinciplesofFinance-TestBank-Ch11.docx
file: 3, PrinciplesofFinance-TestBank-Ch12.docx
file: 4, PrinciplesofFinance-TestBank-Ch13.docx
file: 5, PrinciplesofFinance-TestBank-Ch14.docx
file: 6, PrinciplesofFinance-TestBank-Ch15.docx
file: 7, PrinciplesofFinance-TestBank-Ch16.docx
file: 8, PrinciplesofFinance-TestBank-Ch17.docx
file: 9, PrinciplesofFinance-TestBank-Ch18.docx
file: 10, PrinciplesofFinance-TestBank-Ch19.docx
file: 11, PrinciplesofFinance-TestBank-Ch2.docx
file: 12, PrinciplesofFinance-TestBank-Ch20.docx
file: 13, PrinciplesofFinance-TestBank-Ch3.docx
file: 14, PrinciplesofFinance-TestBank-Ch4.docx
file: 15, PrinciplesofFinance-TestBank-Ch5.docx
file: 16, PrinciplesofFinance-TestBank-Ch6.docx
file: 17, PrinciplesofFinance-TestBank-Ch7.docx
file: 18, PrinciplesofFinance-TestBank-Ch8.docx
file: 19, PrinciplesofFinance-TestBank-Ch9.docx


In [115]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/principles-management/principles-management/*.docx'))
text = docx2txt.process(files[88])
BOOK = 'Principles of Management'

In [116]:
start = re.search("\nConcept Check", text).start()
concepts = text[start:].split('\n\n')[1:]
concepts

['What are the three major types of communication?',
 'How can you manage the inflow of electronic communication?',
 'What are the major influences on organizational communication, and how can organizational design affect communication?',
 'Access for free at https://openstax.org']

In [117]:
question_files = ['critical-thinking-case', 'questions', 'exercises']
def management(files, BOOK):
    print(f"Book: {BOOK}")
    rows = []
    for file in files:
        
        text = docx2txt.process(file)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        is_question = any(q for q in question_files if q in file)
        if is_question:
            if 'exercises' in filename and 'managerial' in filename:
                qs = re.findall('Managerial Decision Exercises\n(.*?)$', text, re.DOTALL)[0].split('\n\n')
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            elif 'exercises' in filename and 'skills' in filename:
                qs = re.findall('Management Skills Application Exercises\n(.*?)$', text, re.DOTALL)[0].split('\n\n')
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            elif 'critical-thinking-case' in filename:
                try:
                    cq = re.findall('Critical Thinking Case\n\n(.*?)$', text, re.DOTALL)[0]
                    qs = re.findall('\n\n.*Questions\n\n(.*?)$', cq, re.DOTALL)[0].split('\n\n')
#                     print(f"success:{qs} \n\n {cq} \n\n")
                except:
                    start = re.search('Questions:', cq).end()
                    end = re.search('Sources:', cq).start()
                    qs = cq[start:end].split('\n\n')
#                     print(f"Critical Thinking: {filename}")
                for q in qs:
                    if len(q) == 0:
                        continue
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = False
                row['text'] = q.strip('\n\n').strip('\n')
                rows.append(row)
            # Last case review Questions
            elif 'questions' in filename:
                qs = re.findall('Review Questions\n\n(.*?)$', text, re.DOTALL)[0].split('\n\n')
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            else:
                print(f"this file is missing: {filename}")
                continue
        else:
#             concepts = re.findall('Concept Check\n\n.*?$', text, re.DOTALL)[0].split('\n\n')[1:]
            try:
                start = re.search("\nConcept Check", text).start()
                concepts = text[start:].split('\n\n')[1:]
                text = text[:start]
            except:
                concepts = []
            for concept in concepts:
                row = dict()
                row['Book'] = BOOK
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = concept
                row['concept'] = 1
#                 print(row)
                rows.append(row)
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = False
            row['text'] = text.strip('\n\n').strip('\n')
            rows.append(row)
    return rows

In [118]:
# df = pd.DataFrame(management(files))
pd.DataFrame(management(files, BOOK)).to_parquet('management_tq.pq')
# flag = df["concept"] == 1
# df[flag].text.unique()[0]

Book: Principles of Management


# Management Review Questions
- very detailed review questions to each section
- each section has a small summary/intro too


In [119]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/OpenStax_PrinciplesofManagement_Review_Questions/OpenStax_PrinciplesofManagement_Review_Questions/*.docx'))
text = docx2txt.process(files[4])
# text

In [120]:
files[0].split('/')[-1].strip('.docx').split('Ch')[-1]

'01'

In [121]:
def management_rq(files, BOOK):
    print(f"Book: {BOOK}")
    rows = []
    for file in files:
        filename = file.split('/')[-1].strip('.docx')
        text = docx2txt.process(file)
        start = re.search("\n\nTo answer a particular question,", text).start()
        text = text[:start]
        chapter = filename.split('Ch')[-1]
        review_questions = re.findall('[1-9]\.[1-9] Review Questions.*?\n\n[1-9]', text, re.DOTALL)
        for section in review_questions:
            for q in section.split('\n\n'):
                if re.match('[1-9]\.[1-9]', q):
                    continue
                else:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = filename
                    row['text'] = q
                    row['is_question'] = True
                    rows.append(row)
    return rows

In [122]:
pd.DataFrame(management_rq(files, BOOK)).to_parquet('management_review_questions.pq')

Book: Principles of Management


# Principles of Microeconomics

- no questions in the text
- each chapter has problems, review questions, self-check questions
- some have critical thinking questions
- We have answers to all self-check questions they are in the chapter files


In [123]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/*.docx'))
text = docx2txt.process(files[3])
BOOK = 'Microeconomics 2e'

In [124]:
files[36]

'/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/12-critical-thinking-questions.docx'

In [125]:
for file in files:
    if 'chapter' in file:
        print(file)

/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-1.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-10.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-11.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-12.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-13.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-14.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-15.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-16.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-17.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-18.docx
/kaggle/input/csnlp-datasets/openstax/openstax/ap-microeconomics-2e-docx/chapter-19.docx
/kaggle/input/csnlp-da

In [126]:
# q = re.findall('\n\n[A-Z].*?\n\n[1-9]', text, re.DOTALL)
# q

In [127]:
question_files = ['problems', 'questions']
def ap_micro(files, BOOK):
    print(f"Book: {BOOK}")
    rows = []
    for file in files:
        if 'chapter' in file:
            #answers to self-check questions
            continue
        if 'references' in file or 'preface' in file:
            continue
        row = dict()
        row['Book'] = BOOK
        text = docx2txt.process(file)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        if 'a' in chapter or 'b' in chapter or 'c' in chapter:
            continue
        is_question = any(q for q in question_files if q in file)
        if is_question:
            if 'review' in filename or 'self-check' in filename or 'critical' in filename:
                qs = re.findall('\n\n[A-Z].*?\n\n', text, re.DOTALL)
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            elif 'problems' in filename:
                qs = re.findall('\n\n[A-Z].*?(?:\n\n[1-9]|\n\nAccess)', text, re.DOTALL)
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
        else:
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = False
            row['text'] = text.strip('\n\n').strip('\n')
            rows.append(row)
    return rows

In [128]:
pd.DataFrame(ap_micro(files, BOOK = BOOK)).to_parquet('microeconomics2e_tq.pq')

Book: Microeconomics 2e


# Microeconomics 2e Testbank

- Has multiple choice and essay questions for each chapter
- some files indicate that some chapters may have true or false, but I couldn't find any and looked through all files. I.e at the end it says how many questions of each type are there and it always says True/False: 0

In [129]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/AP_Micro_2e_Test_Bank/*.docx'))
text = docx2txt.process(files[3])
text[:300]
# BOOK = 'Microeconomics 2e'

'Chapter 4 Test Bank \n\n\n\nMultiple Choice Questions\n\n\n\n\n\n1. The “law of supply” functions in labor markets; that is, a higher ___________ for labor leads to a higher quantity of labor supplied.\n\n\n\nA. price \n\nB. demand\n\nC. supply\n\nD. quantity\n\n\n\nAnswer: A\t\n\n\n\nReference:\n\n\n\nExplanation:\n\n\n\n\n\n2. As the _'

In [130]:
for file in files:
    text = docx2txt.process(file)
    if not re.search('Essay Questions', text):
        print(f'have not: {file}')
    if not re.search('Multiple Choice Questions', text):
        print(f'have not: {file}')
    

In [131]:
# essay = text[essay_start:]
# essay = re.findall('\n\n\n\n[1-9].(.*?)\n\n\n\nExplanation', essay, re.DOTALL)
# # mcq[:3]
# essay

\n\n\n\n1. A tariff differs from a quota in that a tariff is:\n\n\n\nA. levied on imports, whereas a quota is imposed on exports.\n\nB. levied on exports, whereas a quota is imposed on imports.\n\nC. a tax levied on exports, whereas a quota is a limit on the number of units of a good that can be exported.\n\nD. a tax imposed on imports, whereas a quota is an absolute limit to the number of units of a good that can be imported.\n\n\n\nAnswer:

In [132]:
# mcq = text[multiple_start:essay_start]
# mcq = re.findall('\n\n\n\n[1-9].(.*?)\n\n\n\nAnswer', mcq, re.DOTALL)
# mcq[:3]

In [133]:
def micro2e_testbank(files, BOOK):
    print(f"Book: {BOOK}")
    rows = []
    for file in files:
        text = docx2txt.process(file)
        filename = file.split('/')[-1].strip('.docx')
        chapter = filename.split('TestBank-')[-1].split('-')[0].strip('Ch')
        #get multiple choice questions
        multiple_start = re.search('Multiple Choice Questions', text).start() 
        essay_start = re.search('Essay Questions', text).start()
        mcq = text[multiple_start:essay_start]
        mcq = re.findall('\n\n\n\n[1-9].(.*?)\n\n\n\nAnswer', mcq, re.DOTALL)
        #get essay questions
        essay = text[essay_start:]
        if 'Test Bank Statistics Summary' in essay:
            essay_end = re.search('Test Bank Statistics', essay).start()
            essay = essay[:essay_end]
        essay = re.findall('\n\n\n\n[1-9].(.*?)\n\n\n\nExplanation', essay, re.DOTALL)
        for q in mcq:
            q = q.strip(" ")
            fill_blank = False
            if '___' in q:
                fill_blank = True
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['text'] = q.strip('\n\n').strip('\n')
            row['is_question'] = True
            row['truefalse'] = False
            row['fillblank'] = fill_blank
            row['mc'] = True
            rows.append(row)
        for q in essay:
            q = q.strip(" ")
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['text'] = q.strip('\n\n').strip('\n')
            row['is_question'] = True
            row['truefalse'] = False
            row['fillblank'] = False
            row['mc'] = False
            rows.append(row)
    return rows

In [134]:
pd.DataFrame(micro2e_testbank(files, BOOK = BOOK)).to_parquet('micro2e_testbank.pq')

Book: Microeconomics 2e


# Principles of Macroeconomics 2e Testbank
- same as micro eco also no true false, so odd


In [135]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/APMacroEconomics2e-TestBank/APMacroEconomics2e-TestBank/*.docx'))
text = docx2txt.process(files[3])
BOOK = 'Macroeconomics 2e'

In [136]:
files[-1].split('/')[-1].strip('.docx').split('-')[1].strip("TestBankCh")

'16'

In [137]:
def macro2e_testbank(files, BOOK):
    print(f"Book: {BOOK}")
    rows = []
    for file in files:
        text = docx2txt.process(file)
        filename = file.split('/')[-1].strip('.docx')
        if 'TestBankCh' in filename:
            chapter = filename.split('-')[1].strip("TestBankCh")
        else:
            chapter = filename.split('TestBank-')[-1].split('-')[0].strip('Ch')
        #get multiple choice questions
        multiple_start = re.search('Multiple Choice Questions', text).start() 
        essay_start = re.search('Essay Questions', text).start()
        mcq = text[multiple_start:essay_start]
        mcq = re.findall('\n\n\n\n[1-9].(.*?)\n\n\n\nAnswer', mcq, re.DOTALL)
        #get essay questions
        essay = text[essay_start:]
        if 'Test Bank Statistics Summary' in essay:
            essay_end = re.search('Test Bank Statistics', essay).start()
            essay = essay[:essay_end]
        essay = re.findall('\n\n\n\n[1-9].(.*?)\n\n\n\nExplanation', essay, re.DOTALL)
        for q in mcq:
            q = q.strip(" ")
            fill_blank = False
            if '___' in q:
                fill_blank = True
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['text'] = q.strip('\n\n').strip('\n')
            row['is_question'] = True
            row['truefalse'] = False
            row['fillblank'] = fill_blank
            row['mc'] = True
            rows.append(row)
        for q in essay:
            q = q.strip(" ")
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['text'] = q.strip('\n\n').strip('\n')
            row['is_question'] = True
            row['truefalse'] = False
            row['fillblank'] = False
            row['mc'] = False
            rows.append(row)
    return rows

In [138]:
pd.DataFrame(macro2e_testbank(files, BOOK)).to_parquet('macroeconomics2e_testbank')

Book: Macroeconomics 2e


# Principles of MacroEconomics 2e

- same structure as micro 2e, 
- could possibly get rid of the extra files appendices?

In [139]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/ap-macroeconomics-2e-docx/*.docx'))
text = docx2txt.process(files[3])
text

"1.4   How To Organize Economies: An Overview of Economic Systems\n\nLearning Objectives\n\nBy the end of this section, you will be able to:\n\nContrast traditional economies, command economies, and market economies\n\nExplain gross domestic product (GDP)\n\nAssess the importance and effects of globalization\n\nThink about what a complex system a modern economy is. It includes all production of goods and services, all buying and selling, all employment. The economic life of every individual is interrelated, at least to a small extent, with the economic lives of thousands or even millions of other individuals. Who organizes and coordinates this system? Who ensures that, for example, the number of televisions a society provides is the same as the amount it needs and wants? Who ensures that the right number of employees work in the electronics industry? Who ensures that televisions are produced in the best way possible? How does it all get done?\n\nThere are at least three ways that socie

In [140]:
question_files = ['problems', 'questions']
def ap_macro(files, BOOK):
    print(f"book: {BOOK}")
    rows = []
    for file in files:
        if 'chapter' in file:
            #answers to self-check questions
            continue
        if 'references' in file or 'preface' in file:
            continue
        row = dict()
        text = docx2txt.process(file)
        try:
            start = re.search("\nAccess for free", text).start()
            text = text[:start]
        except:
            text = text
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        if 'a' in chapter or 'b' in chapter or 'c' in chapter:
            continue
        is_question = any(q for q in question_files if q in file)
        if is_question:
            if 'review' in filename or 'self-check' in filename or 'critical' in filename:
                qs = re.findall('\n\n[A-Z].*?\n\n', text, re.DOTALL)
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
            elif 'problems' in filename:
                qs = re.findall('\n\n[A-Z].*?(?:\n\n[1-9]|\n\nAccess)', text, re.DOTALL)
                for q in qs:
                    row = dict()
                    row['Book'] = BOOK
                    row['Chapter'] = chapter
                    row['section_title'] = section_title
                    row['is_question'] = True
                    row['text'] = q.strip('\n\n').strip('\n')
                    rows.append(row)
        else:
            row = dict()
            row['Book'] = BOOK
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = False
            row['text'] = text.strip('\n\n').strip('\n')
            rows.append(row)
    return rows

In [141]:
pd.DataFrame(ap_macro(files, BOOK)).to_parquet('macroeconomics2e_tq.pq')

book: Macroeconomics 2e


# Principles of Microeconomics 3e
- same as micro 2e but more content (bigger chapters, also more questions) though much is redundant, may need to filter for duplicates
- questions every chapter, review, critical thinking and self-check questions
- problems almost every chapter

In [142]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Macroeconomics3e_docx/*.docx'))
BOOK = 'Microeconomics 3e'
pd.DataFrame(ap_micro(files, BOOK)).to_parquet('microeconomics3e_tq.pq')

Book: Microeconomics 3e


In [143]:
pd.DataFrame(ap_micro(files, BOOK))

Book: Microeconomics 3e


,Book,Chapter,section_title,is_question,text
0,Microeconomics 3e,1,1-1-what-is-economics-and-why-is-it-important,False,"1.1 What Is Economics, and Why Is It Importa..."
1,Microeconomics 3e,1,1-2-microeconomics-and-macroeconomics,False,1.2 Microeconomics and Macroeconomics\n\nLea...
2,Microeconomics 3e,1,1-3-how-economists-use-theories-and-models-to-...,False,1.3 How Economists Use Theories and Models t...
3,Microeconomics 3e,1,1-4-how-to-organize-economies-an-overview-of-e...,False,1.4 How To Organize Economies: An Overview o...
4,Microeconomics 3e,1,1-critical-thinking-questions,True,Suppose you have a team of two workers: one is...
...,...,...,...,...,...
983,Microeconomics 3e,9,9-self-check-questions,True,How to Measure Changes in the Cost of Living i...
984,Microeconomics 3e,9,9-self-check-questions,True,The Consumer Price Index is subject to the sub...
985,Microeconomics 3e,9,9-self-check-questions,True,Go to this website for the Purchasing Power Ca...
986,Microeconomics 3e,9,9-self-check-questions,True,"If inflation rises unexpectedly by 5%, would a..."


# MicroEconomics Testbank
- mostly the same too
- issues with redundancy again

In [144]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Microeconomics3e_Test_Bank/*.docx'))
text = docx2txt.process(files[3])
text[:300]

'Chapter 4 Test Bank \n\n\n\nMultiple Choice Questions\n\n\n\n\n\n1. The “law of supply” functions in labor markets; that is, a higher ___________ for labor leads to a higher quantity of labor supplied.\n\n\n\nA. price \n\nB. demand\n\nC. supply\n\nD. quantity\n\n\n\nAnswer: A\t\n\n\n\nReference:\n\n\n\nExplanation:\n\n\n\n\n\n2. As the _'

In [145]:
pd.DataFrame(micro2e_testbank(files, BOOK)).to_parquet('micro3e_testbank.pq')

Book: Microeconomics 3e


# Principles of Macroeconomics 3e
- lucky same again

In [146]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Macroeconomics3e_docx/*.docx'))
BOOK = 'Macroeconomics 3e'
pd.DataFrame(ap_macro(files, BOOK)).to_parquet('macroeconomics3e_tq.pq')

book: Macroeconomics 3e


In [147]:
pd.DataFrame(ap_macro(files, BOOK))

book: Macroeconomics 3e


,Book,Chapter,section_title,is_question,text
0,Macroeconomics 3e,1,1-1-what-is-economics-and-why-is-it-important,False,"1.1 What Is Economics, and Why Is It Importa..."
1,Macroeconomics 3e,1,1-2-microeconomics-and-macroeconomics,False,1.2 Microeconomics and Macroeconomics\n\nLea...
2,Macroeconomics 3e,1,1-3-how-economists-use-theories-and-models-to-...,False,1.3 How Economists Use Theories and Models t...
3,Macroeconomics 3e,1,1-4-how-to-organize-economies-an-overview-of-e...,False,1.4 How To Organize Economies: An Overview o...
4,Macroeconomics 3e,1,1-critical-thinking-questions,True,Suppose you have a team of two workers: one is...
...,...,...,...,...,...
983,Macroeconomics 3e,9,9-self-check-questions,True,How to Measure Changes in the Cost of Living i...
984,Macroeconomics 3e,9,9-self-check-questions,True,The Consumer Price Index is subject to the sub...
985,Macroeconomics 3e,9,9-self-check-questions,True,Go to this website for the Purchasing Power Ca...
986,Macroeconomics 3e,9,9-self-check-questions,True,"If inflation rises unexpectedly by 5%, would a..."


In [148]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Macroeconomics3e_TestBank/*.docx'))
pd.DataFrame(macro2e_testbank(files, BOOK)).to_parquet('macroeconomics3e_testbank')

Book: Macroeconomics 3e


In [149]:
pd.DataFrame(macro2e_testbank(files, BOOK))

Book: Macroeconomics 3e


,Book,Chapter,text,is_question,truefalse,fillblank,mc
0,Macroeconomics 3e,01,In countries like _____________ the command ec...,True,False,True,True
1,Macroeconomics 3e,01,In which of the following countries will the n...,True,False,False,True
2,Macroeconomics 3e,01,"In a command economy, the __________ either ma...",True,False,True,True
3,Macroeconomics 3e,01,Because of their relatively small national eco...,True,False,False,True
4,Macroeconomics 3e,01,"In a discussion of economics, which of the fol...",True,False,False,True
...,...,...,...,...,...,...,...
1067,Macroeconomics 3e,21,"“Block Imports—Save Jobs for Some Americans, L...",True,False,False,False
1068,Macroeconomics 3e,21,Define protectionism as a policy and describe ...,True,False,False,False
1069,Macroeconomics 3e,21,"In theory, imports might injure workers in sev...",True,False,False,False
1070,Macroeconomics 3e,21,What is the World Trade Organization and what ...,True,False,False,False


# Economics 3e
- structure is the same as micro and macro but little to no overlap, similar topics though, will add for nice variance
- very large book
- questions every chapter, review, critical thinking and self-check questions
- problems almost every chapter
- no true or false again

In [150]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Economics3e_docx/*.docx'))
BOOK = 'Economics 3e'
df = pd.DataFrame(ap_macro(files, BOOK)).to_parquet('economics3e_tq.pq')

book: Economics 3e


# Economics 3e
- same and again no TF

In [151]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/Economics3e_TestBank/*.docx'))
pd.DataFrame(macro2e_testbank(files, BOOK)).to_parquet('economics3e_testbank.pq')

Book: Economics 3e


# Principles of Marketing
- every section has knowledge-checks
- critical thinking exercises for most chapters
- some chapters have applied marketing questions
- what do marketers do (not really questions but not really lectures either), hybrid, maybe skip
- marketing plan are exercises requiring online files

In [152]:
files = sorted(glob('/kaggle/input/csnlp-datasets/openstax/openstax/principles-marketing_docx/*.docx'))
text = docx2txt.process(files[3])
BOOK = 'Marketing'

In [153]:
# for file in files:
#     if 'what-do-marketers-do' in file:
#         print(file)
#     if 'marketing-plan-exercise' in file:
#         print(file)
for file in files:
    if 'questions' in file:
        text = docx2txt.process(file)
        break

In [154]:
text

'Applied Marketing Knowledge: Discussion Questions\n\n1. \n\nWhy is capturing customer lifetime value so important? Choose a retailer, and apply this concept as a way to explain, in part, its profitability and long-term viability in the marketplace.\n\n2. \n\nThe marketing mix has been compared to a toolbelt. In other words, successful marketers can devise an appropriate marketing mix by combining the right blend of tools: product, price, place, and promotion. Describe a recent product that you purchased on Amazon.com or some other online retailer (place), how you located it (promotion), and whether the price was competitive when compared to similar products.\n\n3. \n\nWhy do some marketers prefer to follow the 4Ps (product, price, place, and promotion) of marketing and others might prefer to follow the new pattern of the 5Ms (minds, minutes, machinery, materials, and money)?\n\n4. \n\nA person can be a product. Please give an example of this.\n\n5. \n\nAccording to the US Centers for 

In [155]:
pattern = r'\n\n(?:.*?)\n\n(?:\d|Access)'
re.findall(pattern, text)

['\n\nWhy is capturing customer lifetime value so important? Choose a retailer, and apply this concept as a way to explain, in part, its profitability and long-term viability in the marketplace.\n\n2',
 '\n\nThe marketing mix has been compared to a toolbelt. In other words, successful marketers can devise an appropriate marketing mix by combining the right blend of tools: product, price, place, and promotion. Describe a recent product that you purchased on Amazon.com or some other online retailer (place), how you located it (promotion), and whether the price was competitive when compared to similar products.\n\n3',
 '\n\nWhy do some marketers prefer to follow the 4Ps (product, price, place, and promotion) of marketing and others might prefer to follow the new pattern of the 5Ms (minds, minutes, machinery, materials, and money)?\n\n4',
 '\n\nA person can be a product. Please give an example of this.\n\n5',
 '\n\nAccording to the US Centers for Disease Control and Prevention, the birthra

In [156]:
files[5]

'/kaggle/input/csnlp-datasets/openstax/openstax/principles-marketing_docx/1-6-customer-relationship-management-crm.docx'

In [157]:
question_files = ['problems', 'questions', 'critical-thinking']
def marketing(files):
    rows = []
    for file in files:
        filename = file.split('/')[-1]
        section_title = re.sub(r"\.docx$", "", filename)
        chapter = filename.split('-')[0]
        if 'chapter' in file:
            #answers to self-check questions
            continue
        if 'what-do-marketers-do' in file or 'marketing-plan-exercise' in file:
            continue
        if 'references' in file or 'preface' in file:
            continue
        row = dict()
        text = docx2txt.process(file)
        if 'critical-thinking' in file:
            pattern = r'\n\n(?:.*?)\n\n(?:\d|Access)'
            questions = re.findall(pattern, text)
            for q in questions:
                row = dict()
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = q.strip('\n\n').strip('\n')
                rows.append(row)
        elif 'questions' in file:
            pattern = r'\n\n(?:.*?)\n\n(?:\d|Access)'
            questions = re.findall(pattern, text)
            for q in questions:
                row = dict()
                row['Chapter'] = chapter
                row['section_title'] = section_title
                row['is_question'] = True
                row['text'] = q.strip('\n\n').strip('\n')
                rows.append(row)
        else:
            try:
                start = re.search("\nAccess for free", text).start()
                text = text[:start]
            except:
                text = text
            row = dict()    
            row['Chapter'] = chapter
            row['section_title'] = section_title
            row['is_question'] = False
            row['text'] = text.strip('\n\n').strip('\n')
            rows.append(row)
    return rows

In [158]:
df = pd.DataFrame(marketing(files))
df['Book'] = BOOK
df = df[['Book', 'Chapter', 'section_title', 'is_question', 'text']]
df.to_parquet('marketing_tq.pq')

In [159]:
df

,Book,Chapter,section_title,is_question,text
0,Marketing,1,1-1-marketing-and-the-marketing-process,False,1.1 Marketing and the Marketing Process\n\nL...
1,Marketing,1,1-2-the-marketing-mix-and-the-4ps-of-marketing,False,1.2 The Marketing Mix and the 4Ps of Marketi...
2,Marketing,1,1-3-factors-comprising-and-affecting-the-marke...,False,1.3 Factors Comprising and Affecting the Mar...
3,Marketing,1,1-4-evolution-of-the-marketing-concept,False,1.4 Evolution of the Marketing Concept\n\nLe...
4,Marketing,1,1-5-determining-consumer-needs-and-wants,False,1.5 Determining Consumer Needs and Wants\n\n...
...,...,...,...,...,...
334,Marketing,9,9-critical-thinking-exercises,True,Explain the product life cycle. How and why mi...
335,Marketing,9,9-critical-thinking-exercises,True,Examine the package of a product that you have...
336,Marketing,9,9-in-the-spotlight,False,Figure 9.1 Marketing strategies define and ...
337,Marketing,9,9-key-terms,False,Key Terms\n\nactual product\n\nproduct availab...


In [160]:
!python --version

Python 3.7.12
